1st attempt: [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 179, 201, 241, 522]

2nd attempt: [276, 394, 301, 595, 404, 311, 355, 404, 437, 394, 524, 357, 393, 374, 501, 237, 509, 356]

In [ ]:
import numpy as np
from skimage import io
from SegmentationStatisticsCollector import convert_cell_labels_to_meshes, process_labels
import trimesh as tm
from skimage import io
from napari_process_points_and_surfaces import label_to_surface
import napari

### Preprocessing of labels:

In [ ]:
img = io.imread("data/segmentation.tif")
img = np.einsum('kij->ijk', img)
image = process_labels(img, dilation_iterations=2, erosion_iterations=4, output_directory='path/to/output/directory', overwrite = False)

In [ ]:
cells = [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 178, 201, 241, 522]
image_path = '/Users/antanas/BC_Project/Control_Segmentation_final/BC_control_s_5_e_2_d_5/processed_labels.npy'

In [ ]:
def create_combined_mesh(cell_labels, image_path):
    # Read the image
    labels = np.load(image_path)
    
    # Initialize an empty mesh
    combined_mesh = None
    labels_list = []
    for label in cell_labels:
        # Create a boolean mask for the current label
        mask = np.array(labels == label).astype(int)

        # Create a mesh from the mask using trimesh
        mesh = convert_cell_labels_to_meshes(mask, voxel_resolution=np.array([0.21, 0.21, 0.39]), smoothing_iterations=100)
        labels_list.append(mesh)

    # Make a combined mesh
    big_mask = np.isin(labels, cell_labels)
    big_mesh = convert_cell_labels_to_meshes(big_mask, voxel_resolution=np.array([0.21, 0.21, 0.39]), smoothing_iterations=100)
    
    return big_mesh, labels_list

combined_mesh, labels_list = create_combined_mesh(cells, image_path)

### Visualise

In [ ]:
vertices, faces = combined_mesh[0].vertices, combined_mesh[0].faces
viewer = napari.Viewer()
viewer.add_surface((vertices, faces), name=f'large_mesh')

In [ ]:
for i, k in enumerate(cells):
    vertices, faces = labels_list[i][0].vertices, labels_list[i][0].faces
    viewer.add_surface((vertices, faces), name=f'mesh {k}')

### Export

In [ ]:
for i, k in enumerate(cells):
    labels_list[i][0].export(f'/Users/antanas/BC_Project/Simulation_Experiment_3/cell_{k-1}.stl')
combined_mesh[0].export(f'/Users/antanas/BC_Project/Simulation_Experiment_3/large_mesh.stl')

### Reading in to visualise

In [ ]:
path = '/Users/antanas/BC_Project/Simulation_Experiment_2'
viewer_1 = napari.Viewer()

for file in os.listdir('/Users/antanas/BC_Project/Simulation_Experiment_2'):
    mesh = tm.load(os.path.join('/Users/antanas/BC_Project/Simulation_Experiment_2', file))
    vertices, faces = mesh.vertices, mesh.faces
    viewer_1.add_surface((vertices, faces), name=os.path.basename(file))

In [ ]:
import numpy as np
import trimesh as tm
from skimage import io
from napari_process_points_and_surfaces import label_to_surface

path = '/Users/antanas/BC_Project/Control_Segmentation/276_mesh_sim.tif'
labels = io.imread(path)
mask = labels * (labels == 276)


mesh_lst = convert_cell_labels_to_meshes(mask, voxel_resolution=np.array([0.21, 0.21, 0.39]), smoothing_iterations=10)

# # Create a boolean array where True corresponds to the label 530
# mask = labels * (labels == 530) == 530

# # Create a mesh from the mask using trimesh
# mesh = label_to_surface(mask)

# print(mesh)



In [ ]:
mesh_lst[0].export('276_mesh.stl')

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()
viewer

In [ ]:
cells = [530, 163, 110, 146, 594, 109, 138, 115, 157, 533, 94, 145, 155, 164, 200, 129, 179, 201, 241, 522]

In [ ]:
import trimesh as tm
mesh_list = []

for i in cells:
    mesh_list.append(tm.load_mesh(f'/Users/antanas/BC_Project/Control_Segmentation_final/BC_control_s_5_e_2_d_5/cell_meshes/cell_{i-1}.stl'))

In [ ]:
viewer = napari.Viewer()
for i, k in enumerate([276, 394, 301, 595, 404, 311, 355, 404, 437, 394, 524, 357, 393, 374, 501, 237, 509, 356]):
    vertices, faces = mesh_list[i].vertices, mesh_list[i].faces
    viewer.add_surface((vertices, faces), name=f'mesh {k}')

In [ ]:
m178 = tm.load_mesh(f'/Users/antanas/BC_Project/Control_Segmentation_final/BC_control_s_5_e_2_d_5/cell_meshes/cell_{177}.stl')
vertices, faces = m178.vertices, m178.faces
viewer.add_surface((vertices, faces), name='mesh 178')

In [ ]:
vertices, faces = mesh_lst[0].vertices, mesh_lst[0].faces
viewer.add_surface((vertices, faces), name='Large Mesh')

In [ ]:
for i, k in enumerate(cells):
    mesh_list[i].export(f'/Users/antanas/BC_Project/Simulation_Experiment/cell_{k-1}.stl')
mesh_lst[0].export(f'/Users/antanas/BC_Project/Simulation_Experiment/large_mesh.stl')

In [ ]:
import os
import napari 

path = '/Users/antanas/BC_Project/Simulation_Experiment_2'
viewer_1 = napari.Viewer()

for file in os.listdir('/Users/antanas/BC_Project/Simulation_Experiment_2'):
    mesh = tm.load(os.path.join('/Users/antanas/BC_Project/Simulation_Experiment_2', file))
    vertices, faces = mesh.vertices, mesh.faces
    viewer_1.add_surface((vertices, faces), name=os.path.basename(file))